Train FFNN model on any type of data, with or without linear preprocessing.<br>
Script based on NNCancellation.py from https://github.com/abalatsoukas/fdnn

In [ ]:
import scipy.io as sio
import numpy as np
import fullduplex as fd
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import pickle
from models import FFNN_model

from config import *

### Script Config

In [ ]:
DATA_TYPE = 'R' # R/H/W
LIN_PREPROCESSING = True

CHANNEL_LEN = 13
TRAINING_RATIO = 0.9

EXPORT_RESULTS = True
PATH_EXPORT_RESULTS = 'comparison_results/new/'

In [ ]:
# This line disables the use of the GPU for training. The dataset is not large enough to get
# significant gains from GPU training and, in fact, sometimes training can even be slower on
# the GPU than on the CPU. Comment out to enable GPU use.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Define system parameters
params = {
		'samplingFreqMHz': 20,	# Sampling frequency, required for correct scaling of PSD
		'hSILen': CHANNEL_LEN,			# Self-interference channel length
		'pamaxordercanc': 7,	# Maximum PA non-linearity order
		'trainingRatio': TRAINING_RATIO,	# Ratio of total samples to use for training
		'dataOffset': 14,		# Data offset to take transmitter-receiver misalignment into account
		'nHidden': 17,			# Number of hidden layers in NN
		'nEpochs': 50,			# Number of training epochs for NN training
		'learningRate': 0.004,	# Learning rate for NN training
		'batchSize': 32,		# Batch size for NN training
		}


### Load Data

In [ ]:
if DATA_TYPE == 'R':

    x, y, noise, measuredNoisePower = fd.loadData(PATH_DATA_REAL, params)

elif DATA_TYPE == 'H':

    matlabVariables = sio.loadmat(PATH_DATA_SYNTH_HAMMERSTEIN+'/fileID0.mat')
    x = np.squeeze(matlabVariables['sig_s'])[:TOTAL_SIGNAL_LENGTH]
    y = np.squeeze(matlabVariables['sig_yH'])[:TOTAL_SIGNAL_LENGTH]
    noise = np.squeeze(np.squeeze(matlabVariables['sig_yH']) - np.squeeze(matlabVariables['sig_x']))

elif DATA_TYPE == 'W':

    matlabVariables = sio.loadmat(PATH_DATA_SYNTH_WIENER+'/fileID0.mat')
    x = np.squeeze(matlabVariables['sig_z'])[:TOTAL_SIGNAL_LENGTH]
    y = np.squeeze(matlabVariables['sig_yW'])[:TOTAL_SIGNAL_LENGTH]
    noise = np.squeeze(np.squeeze(matlabVariables['sig_yH']) - np.squeeze(matlabVariables['sig_x']))

else:
    raise ValueError('DATA_TYPE must be either "R", "H", or "W".')

# remove mean
y = y - np.mean(y)

# Split into training and test sets
trainingSamples = int(np.floor(x.size*params['trainingRatio']))
x_train = x[0:trainingSamples]
y_train = y[0:trainingSamples]
x_test = x[trainingSamples:]
y_test = y[trainingSamples:]

y_test_orig = np.copy(y_test) # for later eval, untouched by lin. preproc.

#### Optional Linear Preprocessing

In [ ]:
if LIN_PREPROCESSING:
    
    # Train-data
    hLin = fd.SIestimationLinear(x_train, y_train, params)
    y_train_lin = fd.SIcancellationLinear(x_train, hLin, params)
    y_train = y_train - y_train_lin

    # Test-data
    y_test_lin = fd.SIcancellationLinear(x_test, hLin, params)
    y_test = y_test - y_test_lin
    

#### Target normalization (regardless of whether lin. preproc. was applied)

In [ ]:
yVar = np.var(y_train)
y_train = y_train/np.sqrt(yVar)
y_test = y_test/np.sqrt(yVar)

### Prepare Model Input

In [ ]:
# Train-data
x_train_real = np.reshape(np.array([x_train[i:i+CHANNEL_LEN].real for i in range(x_train.size-CHANNEL_LEN+1)]), (x_train.size-CHANNEL_LEN+1, CHANNEL_LEN))
x_train_imag = np.reshape(np.array([x_train[i:i+CHANNEL_LEN].imag for i in range(x_train.size-CHANNEL_LEN+1)]), (x_train.size-CHANNEL_LEN+1, CHANNEL_LEN))
x_train = np.zeros((x_train.size-CHANNEL_LEN+1, 2*CHANNEL_LEN))
x_train[:,0:CHANNEL_LEN] = x_train_real
x_train[:,CHANNEL_LEN:2*CHANNEL_LEN] = x_train_imag
y_train = np.reshape(y_train[CHANNEL_LEN-1:], (y_train.size-CHANNEL_LEN+1, 1))

# Test-data
x_test_real = np.reshape(np.array([x_test[i:i+CHANNEL_LEN].real for i in range(x_test.size-CHANNEL_LEN+1)]), (x_test.size-CHANNEL_LEN+1, CHANNEL_LEN))
x_test_imag = np.reshape(np.array([x_test[i:i+CHANNEL_LEN].imag for i in range(x_test.size-CHANNEL_LEN+1)]), (x_test.size-CHANNEL_LEN+1, CHANNEL_LEN))
x_test = np.zeros((x_test.size-CHANNEL_LEN+1, 2*CHANNEL_LEN))
x_test[:,0:CHANNEL_LEN] = x_test_real
x_test[:,CHANNEL_LEN:2*CHANNEL_LEN] = x_test_imag
y_test = np.reshape(y_test[CHANNEL_LEN-1:], (y_test.size-CHANNEL_LEN+1, 1))

### Model Training

In [ ]:
model = FFNN_model(CHANNEL_LEN, params['nHidden'])
adam = Adam(lr=params['learningRate'])
model.compile(loss = "mse", optimizer = adam)

##### Training #####
# Step 2: train NN to do non-linear cancellation
nEpochs = params['nEpochs']
history = model.fit(x_train, [y_train.real, y_train.imag], epochs = nEpochs, batch_size = params['batchSize'], verbose=2, validation_data=(x_test, [y_test.real, y_test.imag]))

##### Test #####
# Do inference step
pred = model.predict(x_test)
y_test_pred = np.squeeze(pred[0] + 1j*pred[1], axis=1) # prev. named "y_test_nl"

In [ ]:
font = {'family' : 'normal',
        'weight' : 'regular',
        'size'   : 14}

mpl.rc('font', **font)


# Calculate various signal powers
noisePower = 10*np.log10(np.mean(np.abs(noise)**2))
scalingConst = PSD_SCALING_CONST#np.power(10,-(measuredNoisePower-noisePower)/10)

if not LIN_PREPROCESSING:
    y_test_lin = 0*y_test_orig

# Plot PSD and get signal powers
fig, noisePower, yTestPower, yTestLinCancPower, yTestNonLinCancPower = fd.plotPSD(
                                                                                y_test_orig[CHANNEL_LEN-1:]/np.sqrt(scalingConst), 
                                                                                y_test_lin[CHANNEL_LEN-1:]/np.sqrt(scalingConst), 
                                                                                y_test_pred/np.sqrt(scalingConst), 
                                                                                noise/np.sqrt(scalingConst), 
                                                                                params, 
                                                                                'NN', 
                                                                                yVar,
                                                                        )

# Print cancellation performance
print('')
print('The linear SI cancellation is: {:.2f} dB'.format(yTestPower-yTestLinCancPower))
print('The non-linear SI cancellation is: {:.2f} dB'.format(yTestLinCancPower-yTestNonLinCancPower))
print('The noise floor is: {:.2f} dBm'.format(noisePower))
print('The distance from noise floor is: {:.2f} dB'.format(yTestNonLinCancPower-noisePower))

In [ ]:

# Plot learning curve
plt.plot(np.arange(1,len(history.history['loss'])+1), -10*np.log10(history.history['loss']), 'bo-')
plt.plot(np.arange(1,len(history.history['loss'])+1), -10*np.log10(history.history['val_loss']), 'ro-')
plt.ylabel('Self-Interference Cancellation (dB)')
plt.xlabel('Training Epoch')
plt.legend(['Training Frame', 'Test Frame'], loc='lower right')
plt.grid(which='major', alpha=0.25)
plt.xlim([ 0, nEpochs+1 ])
plt.xticks(range(1,nEpochs,2))
plt.show()

### Export Results

In [ ]:
if EXPORT_RESULTS:
    file_name = PATH_EXPORT_RESULTS + 'data_' + DATA_TYPE + '_model_FFNN_linSIC_' + ('yes' if LIN_PREPROCESSING else 'no') + '.pkl'
    confirm = input(f'Export as {file_name}? (yes/no)')
    if confirm == 'yes':
        with open(file_name, 'wb') as f:
            pickle.dump({'y_test': y_test_orig[CHANNEL_LEN-1:], 'y_test_lin': y_test_lin[CHANNEL_LEN-1:], 'y_test_nl': y_test_pred, 'noise': noise, 'yVar': yVar, 'chanLen': params['hSILen']}, f)
        print('File saved.')
    else:
        print('File not saved.')